In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter

import wfdb

In [ ]:
allData = []
y = [];

for i in range(45):

    index = '{:03}'.format(i)
    fileName = 'LabWalks/co' + index + '_base'
    if os.path.isfile(fileName + '.hea'):
        record = wfdb.rdrecord(fileName)
        data = record.p_signal
        #tmp = np.linalg.norm(data[:,0:3],axis=1)
        #print(np.size(tmp))
        allData.append(data.T)
        y.append(0)
    
    index = '{:03}'.format(i)
    fileName = 'LabWalks/fl' + index + '_base'
    if os.path.isfile(fileName + '.hea'):
        record = wfdb.rdrecord(fileName)
        data = record.p_signal
        #tmp = np.linalg.norm(data[:,0:3],axis=1)
        #print(np.size(tmp))
        allData.append(data.T)
        y.append(1)

print(np.shape(allData[0]))

In [ ]:
X = np.zeros((len(allData), 12))
for i in range(len(allData)):
    index = 0
    for j in range(6):
        X[i, index] = np.mean(allData[i][j,:])
        index = index + 1
        X[i, index] = np.std(allData[i][j,:])
        index = index + 1

print(X)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xp = scaler.fit_transform(X)

print(Xp)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(Xp)
pca_2d = pca.transform(Xp)


import pylab as pl
for i in range(0, pca_2d.shape[0]):
    if y[i] == 0:
        c1 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='r',    marker='+')
    elif y[i] == 1:
        c2 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='g',    marker='o')

        
pl.legend([c1, c2], ['Nonfaller', 'Faller'])
pl.title('Training dataset with 2 classes')
pl.show()

pca.explained_variance_ratio_

In [ ]:
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)     
    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
        
from sklearn import manifold
#降维
tsne = manifold.TSNE(n_components=2, init='pca', random_state=1)

X_tsne = tsne.fit_transform(Xp)
#绘图
plot_embedding(X_tsne,y*5)
plt.show()

In [ ]:
print(X_tsne)